<a href="https://colab.research.google.com/github/DanyalKanzanAhmad/03_function_calling/blob/main/03_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LangChain: A framework for building applications using language models.
google-generativeai: A library for interacting with Google’s generative AI models (like PaLM and Gemini).
langchain_google_genai: An extension that integrates Google’s generative AI with LangChain.
The -q flag installs them quietly, without showing detailed logs. These libraries allow you to combine LangChain’s tools with Google’s AI models for tasks like text generation and summarization.

In [1]:
!pip install langchain google-generativeai langchain_google_genai -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.6 MB/s eta 0:00:00


This imports the userdata module from Google Colab. The userdata module allows you to store and retrieve information (like API keys) across sessions in a Google Colab environmen

In [66]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
NEWS_API_KEY = userdata.get('News_Api')
WEATHER_API_KEY = userdata.get('Default')
STOCK_API_KEY = userdata.get('Polygon_api')
MOVIE_API_KEY = userdata.get('Omdb_api')


1. calculator Tool:
Purpose: Evaluates a mathematical expression (e.g., "2 + 3 * 4") and returns the result as a float.
Uses Python’s eval() function to evaluate the expression, but restricts access to built-ins for security.
Example: calculator("2 + 3 * 4") returns 14.0.
2. get_stock_price Tool:
Purpose: Fetches the current stock price for a company based on its symbol (e.g., 'AAPL' for Apple).
Uses the Polygon API to get the latest stock price.
Returns a message with the stock price or an error if it fails.
3. get_weather Tool:
Purpose: Fetches the current weather of a specified location (e.g., "New York").
Uses the OpenWeatherMap API to get the weather data.
Returns a summary of the weather, including temperature and conditions.
4. excel_formula_helper Tool:
Purpose: Provides an explanation or usage example of a specific Excel formula (e.g., "VLOOKUP").
Contains a predefined list of formulas and their descriptions.
Returns the explanation for the formula or an error message if not found.
5. get_movie_details Tool:
Purpose: Fetches details about a movie (e.g., title, year, genre) and displays its poster image.
Uses the OMDB API to fetch the movie's details.
Displays the movie's title, year, genre, plot, and poster (if available).
Key Points:
Each tool is decorated with @tool, indicating they are part of a system that uses LangChain for handling tools.
The calculator, get_stock_price, and get_weather tools perform operations based on user input, returning the result.
The get_movie_details tool displays movie information and the movie poster image.
API keys are used for accessing external services like stock data, weather, and movie details.



In [112]:
from langchain_core.tools import tool
import requests
from IPython.display import Image, display


@tool
def calculator(expression: str) -> float:
    """
    Evaluate a mathematical expression and return the result.

    Parameters:
    expression (str): A string containing the mathematical expression to evaluate.

    Returns:
    float: The result of the evaluated expression.

    Examples:
    >>> evaluate_expression("2 + 3 * 4")
    14.0
    >>> evaluate_expression("(10 / 2) + 8")
    13.0

    Note:
    - This function uses Python's `eval()` to calculate the result.
    - Ensure the input is sanitized to avoid malicious code execution.
    """
    try:
        # Evaluate the expression safely
        result = eval(expression, {"__builtins__": {}})
        return float(result)
    except Exception as e:
        print(f"Error evaluating expression: {e}")
        return None


@tool
def get_stock_price(symbol: str) -> str:
    """Fetches the current stock price of a company based on its stock symbol using the Polygon API.

    Args:
        symbol (str): The stock symbol of the company (e.g., 'AAPL' for Apple, 'GOOGL' for Google).

    Returns:
        str: A message containing the current stock price of the company.

    Raises:
        HTTPError: If the HTTP request to the stock API fails (e.g., 404 or 500 status).
        RequestException: If there is an issue with the request itself (e.g., connection error).
        Exception: For any other unexpected errors during the execution of the function.

    """
    api_key = "STOCK_API_KEY"  # Replace this with your actual secret API key from Polygon
    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/prev"  # Polygon endpoint for previous close price

    try:
        # Send a GET request with the API key
        response = requests.get(url, params={'apiKey': api_key})
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)

        # Assuming the data contains 'close' in the response for the last closing price
        data = response.json()
        price = data.get('results', [{}])[0].get('c')  # 'c' is the closing price

        if price:
            return f"Tool used: get_stock_price\n get_stock_price tool is used to find The current price of {symbol} is ${price}"
        else:
            return f"Error: Could not retrieve stock data for {symbol}.\nTool used: get_stock_price"

    except requests.exceptions.HTTPError as http_err:
        return f"HTTP error occurred: {http_err}\nTool used: get_stock_price"
    except requests.exceptions.RequestException as req_err:
        return f"Request error occurred: {req_err}\nTool used: get_stock_price"
    except Exception as err:
        return f"An unexpected error occurred: {err}\nTool used: get_stock_price"

@tool
def get_weather(location: str) -> str:
    """Fetch the current weather for a given location.

     Args:
      location (str): The name of the city or area.

     Returns:
             str: A summary of the weather, including temperature and conditions, or an error message if not found.

     Example:
           >>> get_weather("New York")
       "Weather in New York: Clear skies, 22°C."
    """


    api_key = "WEATHER_API_KEY"  # Replace with your OpenWeatherMap API key
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"

    try:
        response = requests.get(url)
        response.raise_for_status()
        weather = response.json()
        return (f"Weather in {location}: {weather['weather'][0]['description'].capitalize()}, "
                f"{weather['main']['temp']}°C.")
    except Exception as e:
        return f"Error fetching weather: {e}"

@tool
def excel_formula_helper(formula: str) -> str:
    """
    Explain or provide usage for a specific Excel formula.

    Args:
        formula (str): The Excel formula to explain (e.g., "VLOOKUP", "SUMIF").

    Returns:
        str: An explanation or example of the formula usage, or a message if the formula is not found.

    Example:
        >>> excel_formula_helper("VLOOKUP")
        "VLOOKUP(lookup_value, table_array, col_index, [range_lookup]) - Searches for a value."
    """
    formulas = {
        "VLOOKUP": "VLOOKUP(lookup_value, table_array, col_index, [range_lookup]) - Searches for a value in the first column of a range and returns a value in the same row from another column.",
        "SUMIF": "SUMIF(range, criteria, [sum_range]) - Adds values in a range that meet a given condition.",
    }
    return formulas.get(formula.upper(), f"Formula '{formula}' not found in the database.")

@tool
def get_movie_details(title: str) -> None:
    """
    Fetch details about a specific movie and display its poster.

    Args:
        title (str): The title of the movie to search.

    Returns:
        None: Prints the movie details and displays the poster image inline if available.

    Example:
        >>> get_movie_details_with_poster("Inception")
        Title: Inception
        Year: 2010
        Genre: Action, Adventure, Sci-Fi
        Plot: A thief steals corporate secrets...
        (Displays the poster image inline)
    """

    api_key = "e5811495"  # Replace with your OMDB API key
    url = f"http://www.omdbapi.com/?t={title}&apikey={api_key}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        movie = response.json()
        if movie.get("Response") == "False":
            print(f"Movie '{title}' not found.")
            return

        # Display movie details
        print(f"Title: {movie['Title']}")
        print(f"Year: {movie['Year']}")
        print(f"Genre: {movie['Genre']}")
        print(f"Plot: {movie['Plot']}")

        # Display the poster image
        poster_url = movie.get('Poster', 'N/A')
        if poster_url != "N/A":
            display(Image(url=poster_url))
            return "Movie details and poster displayed."
        else:
            return "Movie details displayed, but no poster is available."

    except requests.exceptions.RequestException as req_err:
        return f"Request error: {req_err}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"




This line creates a list called tools which contains references to the functions (or tools) you defined earlier:

calculator: The tool to evaluate mathematical expressions.
get_stock_price: The tool to fetch the current stock price based on the stock symbol.
get_weather: The tool to fetch the current weather for a given location.
excel_formula_helper: The tool to provide explanations or usage for specific Excel formulas.
get_movie_details: The tool to fetch details of a movie and display its poster.
Purpose:
This list is likely used to store the available tools that can be accessed or called later in the code. By organizing them in a list, you can easily iterate over the tools, or choose a specific one based on user input or other conditions.

For example, you could loop through tools and call each function in turn, or randomly pick a tool to use, or even expose this list to the user for interactive access.

If you're planning to work with these tools in a structured way, you can call them based on the context or allow dynamic usage via a command-line interface or API.


In [113]:
tools = [calculator,get_stock_price,get_weather,excel_formula_helper,get_movie_details]

This initializes a ChatGoogleGenerativeAI object, which is responsible for interacting with the Google Gemini model.
The parameters used are:
model="gemini-2.0-flash-exp": Specifies which version of the Gemini model to use (in this case, "gemini-2.0-flash-exp"). You can replace it with other available models if needed.
api_key=GEMINI_API_KEY: The API key used to authenticate requests to the Google Gemini API. The key is fetched from the GEMINI_API_KEY variable, which should contain your Google API key.

In [114]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp" , api_key=GEMINI_API_KEY)

In [118]:
from langchain.agents import initialize_agent, AgentType

In [116]:
agent = initialize_agent(tools, llm , agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION )

In [56]:
respone =  agent.invoke({"input":"what is 2 multiply by 3"})
respone["output"]

'2 multiplied by 3 is 6.'

In [39]:
respone =  agent.invoke({"input":"what is stock price of honda"})
respone["output"]

'The current stock price of Honda (HMC) is $28.65.'

In [40]:
respone =  agent.invoke({"input":"what is the Weather of Sheikhupura"})
respone["output"]

'The weather in Sheikhupura is clear sky with a temperature of 21.32°C.'

In [43]:
respone =  agent.invoke({"input":"what is the sum formula in excel"})
respone["output"]

'The SUM formula in Excel is used to add numbers together. The basic syntax is =SUM(number1, [number2], ...). You can also use ranges of cells, for example, =SUM(A1:A10) would add all the values in cells A1 through A10.'

In [117]:
respone =  agent.invoke({"input":"Bhool Bhulaiyaa"})
respone["output"]

Title: Bhool Bhulaiyaa
Year: 2007
Genre: Comedy, Horror, Mystery
Plot: An NRI and his wife decide to stay in his ancestral home, paying no heed to the warnings about ghosts. Soon, inexplicable occurrences cause him to call a psychiatrist to help solve the mystery.


'I have already displayed the details and poster for the movie Bhool Bhulaiyaa.'